# EfficientNet (Transfer Learning)

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from ptflops import get_model_complexity_info
from torchsummary import summary
import pandas as pd
import time

# Import custom utility module
from utils import get_transform, prepare_train_test_data, calculate_evaluation_metrics

In [3]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

### Load and prepare the data

In [4]:
# Transform data
image_input_size = (3, 256, 256) # Image input size for the model
transform = get_transform(resize_image_size = (256, 256))

# Prepare Train/Val/Test Data
train_loader, val_loader, test_loader = prepare_train_test_data(transform)

# Model keyword
model_keyword = "efficientnet"

### Load pre-trained EfficientNet model and modify it

In [5]:
# Load EfficientNet model
model = models.efficientnet_b0(pretrained=True)

# Modify the classifier for binary classification
num_features = model.classifier[1].in_features
model.classifier = nn.Linear(num_features, 2)  # Binary classification: Internal waves (1) or No waves (0)
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.1) # Implement a learning rate scheduler to adjust the learning rate during training

# Print the summary for EfficientNet
summary(model, input_size=image_input_size, device=str(device))
print(f"Total number of layers: {sum(1 for _ in model.modules())}")

C:\Users\ishar\.virtualenvs\ML_env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\ishar\.virtualenvs\ML_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 128, 128]             864
       BatchNorm2d-2         [-1, 32, 128, 128]              64
              SiLU-3         [-1, 32, 128, 128]               0
            Conv2d-4         [-1, 32, 128, 128]             288
       BatchNorm2d-5         [-1, 32, 128, 128]              64
              SiLU-6         [-1, 32, 128, 128]               0
 AdaptiveAvgPool2d-7             [-1, 32, 1, 1]               0
            Conv2d-8              [-1, 8, 1, 1]             264
              SiLU-9              [-1, 8, 1, 1]               0
           Conv2d-10             [-1, 32, 1, 1]             288
          Sigmoid-11             [-1, 32, 1, 1]               0
SqueezeExcitation-12         [-1, 32, 128, 128]               0
           Conv2d-13         [-1, 16, 128, 128]             512
      BatchNorm2d-14         [-1, 16, 1

### Train the model

In [6]:
# Initialize a DataFrame to log training/validation metrics
log_df = []

# Record the start time
start_time = time.time()

# Training loop
num_epochs = 50
best_val_loss = float('inf')  # Initialize best validation loss for saving the model

for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Validation phase
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = val_correct / val_total * 100
    
    # Adjust learning rate
    scheduler.step(val_loss)
    
    # Calculate FLOPs after each epoch
    with torch.no_grad():
        flops, _ = get_model_complexity_info(model, image_input_size, as_strings=False, print_per_layer_stat=False)

    # Log metrics into the DataFrame
    new_row = {
        "Epoch": epoch + 1,
        "Train_Loss": train_loss / len(train_loader),
        "Validation_Loss": val_loss / len(val_loader),
        "Validation_Accuracy": val_accuracy,
        "FLOPs": flops
    }
    log_df.append(new_row)

    # Print metrics for this epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss / len(train_loader):.4f}, "
          f"Val Loss: {val_loss / len(val_loader):.4f}, Val Accuracy: {val_accuracy:.2f}%, "
          f"GFLOPs: {(flops / 1e9 ):.2f}")

    # Save the best model based on validation loss
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), f"model_outputs_data/best_{model_keyword}_model.pth")

# Calculate the total training time in seconds
print(f"\nTotal Training Time: {(time.time() - start_time):.2f} seconds")

# Save the DataFrame to a CSV file for later use
log_df = pd.DataFrame(log_df)
log_df.to_csv(f"model_outputs_data/model_evaluation_logs/training_logs_{model_keyword}.csv", index=False)

Epoch [1/50], Train Loss: 0.3566, Val Loss: 0.3588, Val Accuracy: 84.73%, GFLOPs: 0.53
Epoch [2/50], Train Loss: 0.2351, Val Loss: 0.1908, Val Accuracy: 91.92%, GFLOPs: 0.53
Epoch [3/50], Train Loss: 0.1480, Val Loss: 0.2703, Val Accuracy: 92.22%, GFLOPs: 0.53
Epoch [4/50], Train Loss: 0.1469, Val Loss: 0.1928, Val Accuracy: 93.83%, GFLOPs: 0.53
Epoch [5/50], Train Loss: 0.1134, Val Loss: 0.1743, Val Accuracy: 94.13%, GFLOPs: 0.53
Epoch [6/50], Train Loss: 0.0787, Val Loss: 0.1978, Val Accuracy: 92.95%, GFLOPs: 0.53
Epoch [7/50], Train Loss: 0.0805, Val Loss: 0.1622, Val Accuracy: 94.86%, GFLOPs: 0.53
Epoch [8/50], Train Loss: 0.0433, Val Loss: 0.1363, Val Accuracy: 95.01%, GFLOPs: 0.53
Epoch [9/50], Train Loss: 0.0287, Val Loss: 0.1498, Val Accuracy: 95.45%, GFLOPs: 0.53
Epoch [10/50], Train Loss: 0.0418, Val Loss: 0.1151, Val Accuracy: 96.04%, GFLOPs: 0.53
Epoch [11/50], Train Loss: 0.0432, Val Loss: 0.2488, Val Accuracy: 91.48%, GFLOPs: 0.53
Epoch [12/50], Train Loss: 0.0340, Val Lo

### Prediction and Model evaluation

In [7]:
model.eval()
# Load the best model for testing
# model.load_state_dict(torch.load(f"model_outputs_data/best_{model_keyword}_model.pth"))

# Test the model
all_labels = []
all_preds = []
all_probs = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        probabilities = torch.softmax(outputs, dim=1)
        _, predicted = torch.max(outputs, 1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())
        all_probs.extend(probabilities[:, 1].cpu().numpy())

# Print evaluation metrics
calculate_evaluation_metrics(all_labels, all_preds, all_probs, model_keyword)


Test Accuracy: 96.37%
Precision: 0.98
Recall: 0.95
F1-Score: 0.96

Classification Report:
                   precision    recall  f1-score   support

No Internal Waves       0.95      0.98      0.96       719
   Internal Waves       0.98      0.95      0.96       740

         accuracy                           0.96      1459
        macro avg       0.96      0.96      0.96      1459
     weighted avg       0.96      0.96      0.96      1459


Confusion Matrix:
[[703  16]
 [ 37 703]]

 Predictions saved to model_outputs_data/model_prediction_logs/efficientnet_labels_predictions.csv
